Importing the required libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import PorterStemmer
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import files

import io
import missingno
import seaborn as sns

Loading the Dataset

In [ ]:
uploaded = files.upload()

Saving Climate_twitter.csv to Climate_twitter.csv


In [ ]:
data =  pd.read_csv(io.BytesIO(uploaded['Climate_twitter.csv']))

In [ ]:
# Data['word_counts'] = Data['message'].str.split().str.len()
# Data["Text Length"] = Data["message"].str.len()
# Data.groupby('sentiment')['word_counts'].mean()

# Exploratory analysis
data.describe()
print(data.columns)
data.head()

# print(Data["message"])
# Data Visualization
# sns.histplot(data=Data, x="sentiment", binwidth=0.4, color='lime')


Index(['id', 'date', 'retweets', 'source', 'author', 'likes', 'text',
       'twitter_name', 'location', 'verified', 'followers', 'friends',
       'polarity', 'subjectivity'],
      dtype='object')


,id,date,retweets,source,author,likes,text,twitter_name,location,verified,followers,friends,polarity,subjectivity
0,2184934963,2020-12-22 23:22:20,71,Twitter Web App,GO GREEN,91,The death of summer Arctic ice our Earth coole...,ECOWARRIORSS,NaN,False,23415,20439,-0.054365,0.426984
1,508658626,2020-12-10 14:30:00,14,Twitter for Advertisers,Elsevier Energy,98,Elsevier and the EditorsinChief are pleased to...,ElsevierEnergy,"Oxford, England",False,6615,508,0.387500,0.633333
2,2607105006,2020-12-22 21:28:52,0,Twitter Web App,Arwyn Thomas,1,From better climate change education to improv...,siwarr5,Carmarthen,False,22,133,0.261905,0.345238
3,19609660,2020-12-22 21:24:10,0,Twitter Web App,"Tom Gillispie, EDITOR/WRITER",0,climate change Links to FIXING CLIMATE CHANGE ...,EDITORatWORK,"Rural Hall, North Carolina, USA",False,4191,3708,0.000000,0.000000
4,19609660,2020-12-21 22:52:09,1,Twitter Web App,"Tom Gillispie, EDITOR/WRITER",1,climate change The 11TH HOUR FOR THE EARTH cli...,EDITORatWORK,"Rural Hall, North Carolina, USA",False,4191,3708,0.000000,0.000000


In [ ]:
# Checking for missing values
data.isna().sum()

id                0
date              0
retweets          0
source            0
author            0
likes             0
text              0
twitter_name      0
location        103
verified          0
followers         0
friends           0
polarity          0
subjectivity      0
dtype: int64

In [ ]:
# Cleaning the data
def msg_cleaning(msg):
    # Removing @abc12
    msg = re.sub(r'@[A-Za-z0-9]+', '', msg)
    # Removing Hashtags
    msg = re.sub(r'#', '', msg)
    # Removing Chines
    msg = re.sub(r'[^\x00-\x7F]+', '', msg)
    # Removing Retweets
    msg = re.sub(r'RT[\s]+', '', msg)
    msg = re.sub(r'rt[\s]+', '', msg)
    # Removing HyperLinks
    msg = re.sub(r'https?:\/\/\s+', '', msg)
    # Removing numeric values
    msg = re.sub(r'\d+', '', msg)
    msg = re.sub(r'aa[A-Za-z0-9]+', '', msg)
    msg = re.sub(r'zz[A-Za-z0-9]+', '', msg)
    return msg

data['text'] = data['text'].apply(msg_cleaning)
data["text"] = data["text"].str.lower()
# print(Data["message"])

In [ ]:
def identify_tokens(row):
    ide_words = row["text"]
    tokens = word_tokenize(ide_words)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words 

data["text"] = data.apply(identify_tokens, axis=1)
print(data['text'])

stemming = PorterStemmer()

def stem_list(row):
    my_list = row["text"]
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)


0      [the, death, of, summer, arctic, ice, our, ear...
1      [elsevier, and, the, editorsinchief, are, plea...
2      [from, better, climate, change, education, to,...
3      [climate, change, links, to, fixing, climate, ...
4      [climate, change, the, th, hour, for, the, ear...
                             ...                        
391    [an, inconsistent, truth, from, global, coolin...
392    [climate, thought, alongside, einstein, robeop...
393    [climate, used, as, supporting, evidence, that...
394    [global, warming, global, government, via, you...
395    [its, feared, that, climate, change, will, mak...
Name: text, Length: 396, dtype: object


In [ ]:

data["text"] = data.apply(stem_list, axis=1)
print(data["text"])

stops = set(stopwords.words("english"))
stops.update(["aa", "aaa", "aaaa", "aaaaa", "aaaaaa", "aaaaaaa", "aaaaaaaa", "aaaaaaaaa", "aaaaaaaaaaaaaaaaaaaah"])


def remove_stops(row):
    my_list = row["text"]
    meningful_words = [w for w in my_list if not w in stops]
    return(meningful_words)


data["text"] = data.apply(remove_stops, axis=1)
print(data["text"])

data.to_csv("SEMI.csv")


0      [the, death, of, summer, arctic, ice, our, ear...
1      [elsevi, and, the, editorsinchief, are, pleas,...
2      [from, better, climat, chang, educ, to, improv...
3      [climat, chang, link, to, fix, climat, chang, ...
4      [climat, chang, the, th, hour, for, the, earth...
                             ...                        
391    [an, inconsist, truth, from, global, cool, s, ...
392    [climat, thought, alongsid, einstein, robeoppe...
393    [climat, use, as, support, evid, that, man, ma...
394    [global, warm, global, govern, via, youtub, cl...
395    [it, fear, that, climat, chang, will, make, ho...
Name: text, Length: 396, dtype: object
0      [death, summer, arctic, ice, earth, cooler, ye...
1      [elsevi, editorsinchief, pleas, share, first, ...
2      [better, climat, chang, educ, improv, sustain,...
3      [climat, chang, link, fix, climat, chang, clim...
4      [climat, chang, th, hour, earth, climatechang,...
                             ...                 

**Reading the datasets**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn import preprocessing
from google.colab import files
#----
import io
import missingno
import seaborn as sns

In [ ]:
uploaded = files.upload()

Saving First_processed.csv to First_processed.csv


In [ ]:
#Loading Dataset
data = pd.read_csv(io.BytesIO(uploaded['First_processed.csv']))
X = data["message"]
Y = data["sentiment"]

In [ ]:
uploaded = files.upload()

Saving SEMI.csv to SEMI (1).csv


In [ ]:
val =  pd.read_csv(io.BytesIO(uploaded['SEMI.csv']))
x_val = val["text"]

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=20000, use_idf=True)
tfidf.fit_transform(X)
tfidf.fit_transform(x_val)

X = tfidf.transform(X)
MinMaxScaler = preprocessing.Normalizer()

X = MinMaxScaler.fit_transform(X)
x_val = tfidf.transform(x_val)
x_val = MinMaxScaler.fit_transform(x_val)


In [ ]:
#Fitting the model to the data
svm = SVC(C=10, gamma=1, kernel='rbf', verbose=True)
svm.fit(X, Y)

[LibSVM]

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [ ]:
#Saving the predictions/labels for our unlabelled data
predictions = svm.predict(x_val)
val["sentiment"] = predictions
val.to_csv("SEMI_PREDICTED.csv")

**Importing the modules and reading the datasets**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,\
    RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix,\
     f1_score, precision_score, recall_score, accuracy_score
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
import os, sys
from scipy import stats
import numpy as np

In [ ]:
uploaded = files.upload()

Saving First_processed.csv to First_processed (1).csv


In [ ]:
data = pd.read_csv(io.BytesIO(uploaded['First_processed.csv']))

In [ ]:
uploaded = files.upload()

Saving SEMI_PREDICTED.csv to SEMI_PREDICTED (1).csv


In [ ]:
val = pd.read_csv(io.BytesIO(uploaded['SEMI_PREDICTED.csv']))
Data_Set = pd.concat([data, val])
X = Data_Set["message"]
Y = Data_Set["sentiment"]

In [ ]:
#Splitting the Dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=1103)
#splitting the data into validation
X_test, x_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5,shuffle=True, random_state=1103)

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=20000, use_idf=True)
tfidf.fit_transform(X_train)
tfidf.fit_transform(x_val)

X_train = tfidf.transform(X_train)
x_val = tfidf.transform(x_val)
MinMaxScaler = preprocessing.Normalizer()

X_train = MinMaxScaler.fit_transform(X_train)
x_val = MinMaxScaler.fit_transform(x_val)

In [ ]:
svm = SVC(C=10, gamma=1, kernel='rbf', decision_function_shape='ovr', verbose=True)
svm.fit(X_train, y_train)

[LibSVM]

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [ ]:
#Evaluating the model
prediction = svm.predict(x_val)
print("ACCURACY SCORE:", metrics.accuracy_score(y_val, prediction))
print("::::Confusion Matrix::::")
print(confusion_matrix(y_val, prediction))
print("\n")

print(":::Classification Report:::")
print(classification_report(y_val, prediction, target_names=['Class 1', 'Class 2', 'Class 3', 'Class 4']))
print("\n")

print(pd.crosstab(y_val, prediction, rownames=["Orgnl"], colnames=['Predicted']))


ACCURACY SCORE: 0.7201172755976545
::::Confusion Matrix::::
[[ 163   54  179   22]
 [  19  364  368   55]
 [  16  116 2017  113]
 [   5   26  268  649]]


:::Classification Report:::
              precision    recall  f1-score   support

     Class 1       0.80      0.39      0.52       418
     Class 2       0.65      0.45      0.53       806
     Class 3       0.71      0.89      0.79      2262
     Class 4       0.77      0.68      0.73       948

    accuracy                           0.72      4434
   macro avg       0.73      0.60      0.64      4434
weighted avg       0.72      0.72      0.71      4434



Predicted   -1    0     1    2
Orgnl                         
-1         163   54   179   22
 0          19  364   368   55
 1          16  116  2017  113
 2           5   26   268  649
